In [ ]:
import cv2
import albumentations as A
from matplotlib import pyplot as plt
import plotly.express as px
import json

In [ ]:
# stuff for u_c2s_half_odom_stereo120.jpg
bbox = [493.0, 455.0, 50.0, 29.0]
segmentation = [[493.3, 484.5, 543.7, 484.2, 536.2, 455.7, 501.9, 455.8]]

In [ ]:
def load_image(image_path):
    img = imageio.imread(image_path)
    img = ia.imresize_single_image(img, 1.0)
    return img

def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data

def get_annotation(json_path):
    f = load_json(json_path)
    return f["shapes"][0]["points"]
    

def modify_json(json_path, orig_json, polygons, img_path):
    data = load_json(orig_json)
    data['shapes'] = polygons
    data['imagePath'] = img_path
    with open(json_path, 'w') as f:
        json.dump(data, f)
        
def aug(image, polygons):
    # Get correct polygon format
    psoi = ia.PolygonsOnImage([polygons], shape=image.shape)
    
    # Create augmentation pipeline
    aug = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=10),
    # iaa.CoarseDropout(0.1, size_px=8),
    iaa.AddToHueAndSaturation((-50, 50))
    ])

    # Run augmentation
    image_aug, psoi_aug = aug(image=image, polygons=psoi)
    return image_aug, psoi_aug

In [ ]:
img_path = '/home/user/rosbags/final/object_detection/u_c2s_half_odom_stereo120.jpg'
json_path = '/home/user/rosbags/final/object_detection/u_c2s_half_odom_stereo120.json'

# Load the image
img = load_image(img_path)
# Load the annotations
poly = get_annotation(json_path)



In [ ]:
poly

In [ ]:
f["shapes"][0]["points"]

In [ ]:
f["shapes"]
# f["imagePath"]

In [ ]:
def augmentate(img):
    

In [ ]:
import imageio
import imgaug as ia
%matplotlib inline

# image = cv2.imread("/home/user/rosbags/final/object_detection/u_c2s_half_odom_stereo120.jpg", cv2.COLOR_RGB2BGR)
image = imageio.imread("/home/user/rosbags/final/object_detection/u_c2s_half_odom_stereo120.jpg")
image = ia.imresize_single_image(image, 1.0)
print(image.shape)

px.imshow(image)

In [ ]:
import numpy as np
from imgaug.augmentables.polys import Polygon, PolygonsOnImage
# Transform bbox to imgaug format
ramp = Polygon([
    (493.3, 484.5), 
    (543.7, 484.2),
    (536.2, 455.7),
    (501.9, 455.8)
])

image_polys = np.copy(image)
image_polys = ramp.draw_on_image(image_polys, alpha_face=0.2, size_points=3)
px.imshow(image_polys)

In [ ]:
image_polys = ramp.to_bounding_box().draw_on_image(image_polys, size=3)
px.imshow(image_polys)

In [ ]:
psoi = ia.PolygonsOnImage([ramp], shape=image.shape)

In [ ]:
import imgaug.augmenters as iaa
ia.seed(42)

aug = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=10),
    # iaa.CoarseDropout(0.1, size_px=8),
    iaa.AddToHueAndSaturation((-50, 50))
])

image_aug, psoi_aug = aug(image=image, polygons=psoi)
px.imshow(psoi_aug.draw_on_image(image_aug, alpha_face=0.2, size_points=7))

In [ ]:
# Declare an augmentation pipeline
transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], 
    bbox_params=A.BboxParams(format="coco", label_fields=["bbox_classes"]),)

# Read an image with OpenCV and convert it to the RGB colorspace
image = cv2.imread("/home/user/rosbags/final/object_detection/u_c2s_half_odom_stereo120.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Augment an image
transformed = transform(image=image, bboxes=bbox)
transformed_image = transformed["image"]

plt.imshow(image)
plt.show()

plt.imshow(transformed_image)
plt.show()